<a href="https://colab.research.google.com/github/IvanStukalov/MLTechnologies/blob/lab_6/lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа 6. Ансамбли моделей машинного обучения. Часть 2.

In [ ]:
!pip install heamy

In [53]:
from sklearn.datasets import load_wine, load_iris
from sklearn.model_selection import train_test_split
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
from heamy.dataset import Dataset
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score


### Выберите набор данных (датасет) для решения задачи классификации или регресии.

In [39]:
# Используем датасет wine
wine = load_wine()
wine_X = wine.data
wine_y = wine.target

### С использованием метода train_test_split разделите выборку на обучающую и тестовую.

In [40]:
wine_X_train, wine_X_test, wine_y_train, wine_y_test = train_test_split(
    wine_X, wine_y, test_size=0.2, random_state=1)

### Стекинг

In [46]:
# Используем библиотеку heamy
# набор данных
dataset = Dataset(wine_X_train, wine_y_train, wine_X_test)

# модели первого уровня
model_tree = Regressor(dataset=dataset, estimator=DecisionTreeRegressor, name='tree')
model_lr = Regressor(dataset=dataset, estimator=LinearRegression, name='lr')
model_rf = Regressor(dataset=dataset, estimator=RandomForestRegressor, parameters={'n_estimators': 50}, name='rf')

In [47]:
# Эксперимент 1
# Первый уровень - две модели: дерево и линейная регрессия
# Второй уровень: линейная регрессия

pipeline = ModelsPipeline(model_tree, model_lr)
stack_ds = pipeline.stack(k=10, seed=1)
# модель второго уровня
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression)
results = stacker.validate(k=10,scorer=mean_absolute_error)

Metric: mean_absolute_error
Folds accuracy: [0.1449944828491955, 0.2367813603542328, 0.16366879153700514, 0.1734374337213063, 0.22276475695752668, 0.18507815651883502, 0.3385973871111662, 0.1327745371838229, 0.15668457067108263, 0.22931197617907678]
Mean accuracy: 0.198409345308325
Standard Deviation: 0.057951283593480486
Variance: 0.0033583512701320004


### модель многослойного персептрона

In [57]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Создание датасета
# X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)

# Разделение данных на обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение параметров MLP
mlp_params = {
    'hidden_layer_sizes': (50, 50), # Количество нейронов в скрытых слоях
    'activation': 'relu', # Функция активации
    'solver': 'adam', # Оптимизатор
    'alpha': 0.0001, # Коэффициент регуляризации L2
    'learning_rate': 'constant', # Стратегия изменения скорости обучения
    'learning_rate_init': 0.001, # Начальная скорость обучения
    'max_iter': 200, # Максимальное количество итераций
    'random_state': 42, # Случайное состояние для воспроизводимости
}

# Создание и обучение MLP
mlp = MLPClassifier(**mlp_params)
mlp.fit(wine_X_train, wine_y_train)

# Предсказание на тестовом наборе данных
y_pred = mlp.predict(wine_X_test)

# Оценка качества модели
accuracy = accuracy_score(wine_y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8611111111111112


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### МГУА

In [70]:
from gmdhpy import gmdh

ERROR: Invalid requirement: 'git+git@github.com:kvoyager/GmdhPy.git'
Hint: It looks like a path. File 'git+git@github.com:kvoyager/GmdhPy.git' does not exist.


In [65]:
model = gmdh.Regressor()
model.fit(wine_X_train, wine_y_train)
y_pred = model.predict(wine_X_test)
result = mean_absolute_error(wine_y_test, y_pred)
print(model)
print('MAE={}'.format(result))

AttributeError: module 'gmdhpy.gmdh' has no attribute 'Regressor'